In [ ]:
#| default_exp metric.discrete

# DiscreteMetric
> Base class from which all discrete metrics should inherit.

In [ ]:
#| export
import typing as t
from dataclasses import dataclass, field
from pydantic import BaseModel, create_model
from collections import Counter
from ragas_annotator.metric import Metric, MetricResult
from ragas_annotator.metric.decorator import create_metric_decorator


@dataclass
class DiscreteMetric(Metric):
    values: t.List[str] = field(default_factory=lambda: ["pass", "fail"])
    
    def _get_response_model(self, with_reasoning: bool) -> t.Type[BaseModel]:
        """Get or create a response model based on reasoning parameter."""
        
        if with_reasoning in self._response_models:
            return self._response_models[with_reasoning]
        
        model_name = 'response_model'
        values = tuple(self.values)
        fields = {"result": (t.Literal[values], ...)}
        
        if with_reasoning:
            fields["reason"] = (str, ...) # type: ignore
        
        model = create_model(model_name, **fields)  # type: ignore
        self._response_models[with_reasoning] = model
        return model 

    def _ensemble(self,results:t.List[MetricResult]) -> MetricResult:


        if len(results)==1:
            return results[0]
            
        candidates = [candidate.result for candidate in results]
        counter = Counter(candidates)
        max_count = max(counter.values())
        for candidate in results:
            if counter[candidate.result] == max_count:
                result = candidate.result              
                reason = candidate.reason
                return MetricResult(result=result, reason=reason)
        
        return results[0]


discrete_metric = create_metric_decorator(DiscreteMetric)

/opt/homebrew/Caskroom/miniforge/base/envs/random/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Example usage

In [ ]:
from dotenv import load_dotenv

# Load environment variables from a .envrc file
load_dotenv('/Users/shahules/Myprojects/ragas_annotator/.envrc')

True

In [ ]:

#| eval: false

from ragas_annotator.llm import ragas_llm
from openai import OpenAI

llm = ragas_llm(provider="openai",model="gpt-4o",client=OpenAI())


my_metric = DiscreteMetric(
    llm=llm,
    name='helpfulness',
    prompt="Evaluate if given answer is helpful\n\n{response}",
    values=["low","med","high"],
)

result = my_metric.score(response="this is my response")
print(result) #gives "low"
print(result.reason) #gives reasoning from llm



low
The response does not provide any specific information or context that can help evaluate its helpfulness.


### Write custom discrete metric

In [ ]:
#| eval: false
@discrete_metric(llm=llm,
    prompt="Evaluate if given answer is helpful\n\n{response}",
    name='new_metric',values=["low","med","high"])
def my_metric(llm,prompt,**kwargs):

        class response_model(BaseModel):
             output: t.List[bool]
             reason: str
        
        response = llm.generate(prompt.format(**kwargs),response_model=response_model)
        total = sum(response.output)
        if total < 1:
            score = 'low'
        else:
            score = 'high'
        return score,"reason"

result = my_metric.score(response='my response') # result
print(result)
print(result.reason)

high
reason
